## AutoGluon Demos

### 📦 Setup: AutoGluon Environment
This cell installs the minimal dependencies (`autogluon`).
It also creates folders to store models and outputs so the workflow can run start-to-finish in Colab.

In [8]:
# --- Colab setup: AutoGluon Tabular ---
import os

# ---- CONFIG ----
DATA_DIR = "/content/data"
AUTOGLUON_SAVE_PATH = os.path.join(DATA_DIR, "AutoGluonModels")

print("AUTOGLUON_SAVE_PATH:", AUTOGLUON_SAVE_PATH)

# ---- Slim install AutoGluon ----
!python -m pip install -q autogluon.tabular

AUTOGLUON_SAVE_PATH: /content/data/AutoGluonModels


### Load Data

For this experiment, we use a dataset featured in the cover story of Nature issue 7887, “AI-guided intuition for mathematical theorems.” The task is to predict a knot’s signature from its various properties.

The sample contains 10,000 training examples and 5,000 test examples from the original dataset to keep training lightweight. A fixed random seed is also defined to support reproducibility. However, when using AutoGluon, perfect bit-for-bit reproducibility cannot be guaranteed — especially for models leveraging GPUs or parallel computation, where floating-point operations can introduce small numerical differences between runs.

In [9]:
# === Tabular Dataset ===
import numpy as np, random
from autogluon.tabular import TabularDataset

# ---- Reproducible seed ----
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Propagate a seed into AutoGluon’s fitting stage
ag_args_fit = {"random_seed": SEED}

data_url = "https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/"
train_data = TabularDataset(f"{data_url}train.csv")
train_data.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/train.csv | Columns = 19 / 19 | Rows = 10000 -> 10000


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


Here we are showing that our target data is in "signature" where we have a categorical data type.

In [10]:
label = 'signature'
train_data[label].describe()

,signature
count,10000.000000
mean,-0.022000
std,3.025166
min,-12.000000
25%,-2.000000
50%,0.000000
75%,2.000000
max,12.000000


### Training

In this step, we define a save path for our AutoGluon models and create a TabularPredictor object. This object serves as the main interface for training and evaluating models. We then call its .fit() method, which automatically selects and trains the best-performing models for our dataset.

In [11]:
# === Tabular Predictor ===
import time
from autogluon.tabular import TabularPredictor

# --- Use a fresh, unique run folder to avoid "path already exists" warning ---
BASE_PATH = AUTOGLUON_SAVE_PATH
os.makedirs(BASE_PATH, exist_ok=True)
RUN_NAME = f"california_house_{time.strftime('%Y%m%d_%H%M%S')}"
MODEL_PATH = os.path.join(BASE_PATH, RUN_NAME)

predictor = TabularPredictor(
    label=label,
    path=MODEL_PATH
)

predictor.fit(
    train_data=train_data,
    presets="medium",
    time_limit=60,
    ag_args_fit=ag_args_fit,
)

Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.28 GB / 12.67 GB (89.0%)
Disk Space Avail:   186.15 GB / 225.83 GB (82.4%)
Presets specified: ['medium']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/content/data/AutoGluonModels/california_house_20251015_150637"
Train Data Rows:    10000
Train Data Columns: 18
Label Column:       signature
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 13) unique label values:  [np.int64(-2), np.int64(0), np.int64(2), np.int64(-8), np.int64(4), np.int64(-4), np.int

### Prediction

Now that we have a predictor, we can load our test data for prediction.

In [12]:
test_data = TabularDataset(f"{data_url}test.csv")

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


,signature
0,-4
1,-2
2,0
3,4
4,2


### Evaluation

Having generated predictions on the test dataset, we can now evaluate our model’s performance. AutoGluon provides detailed evaluation to summarize how well the trained models generalize to unseen data.

In [13]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9476,
 'balanced_accuracy': np.float64(0.7526070835141294),
 'mcc': np.float64(0.9358079111742555)}

AutoGluon allows us to view a detailed leaderboard that highlights the performance of the various models compared to one another, to understand which models performed better than others.

In [14]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9476,0.961962,accuracy,2.439880,0.286474,26.223441,0.006514,0.004343,0.365417,2,True,9
1,LightGBM,0.9456,0.955956,accuracy,1.103356,0.144355,7.290405,1.103356,0.144355,7.290405,1,True,3
2,RandomForestEntr,0.9384,0.949950,accuracy,0.504663,0.132838,8.394397,0.504663,0.132838,8.394397,1,True,5
3,ExtraTreesGini,0.9360,0.946947,accuracy,1.199570,0.122413,3.064949,1.199570,0.122413,3.064949,1,True,6
4,ExtraTreesEntr,0.9358,0.942943,accuracy,1.072720,0.130924,2.546197,1.072720,0.130924,2.546197,1,True,7
5,RandomForestGini,0.9352,0.944945,accuracy,0.658763,0.107966,7.945950,0.658763,0.107966,7.945950,1,True,4
6,LightGBMXT,0.9320,0.945946,accuracy,2.202644,0.278285,8.107871,2.202644,0.278285,8.107871,1,True,2
7,NeuralNetFastAI,0.9304,0.946947,accuracy,0.130440,0.015363,15.502670,0.130440,0.015363,15.502670,1,True,1
8,XGBoost,0.9220,0.933934,accuracy,0.149094,0.040172,2.204570,0.149094,0.040172,2.204570,1,True,8


### Conclusion

This was a simple example of a tabular prediction using AutoGluon. This highlights the value of AutoML in being able to use a single interface to compare the performance of multiple models with little code required.